# Questions
1. What tables and features differentiate blood-transfusion-related and non-blood-transfusion-related admittances?
2. How many hospital admittances in MIMIC-III are for blood transfusions?
3. Break down these stats by race, by gender, by other demographics
4. How common is it for a blood transfusion to be cut short?
5. What's the average/most extreme body temp difference between the start of transfusion end of transfusion? After 2 hrs? After 4? 8? 24?
6. Characterise the clinicians notes for admittances that were for blood transfusions.

In [86]:
import os
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
pd.set_option('display.max_rows', None)    # Show all rows



load_dotenv()
query_path = os.environ.get('BASE_QUERY_PATH')
client = bigquery.Client(os.environ.get('BIGQUERY_PROJECT_NAME'))

In [96]:
with open(f"{query_path}/scratch.sql", 'r') as file:
    query = file.read()

results = client.query(query).to_dataframe()
results

/home/kmorhun/.pyenv/versions/3.12.7/envs/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,itemid,label,category,dbsource,count
0,5649,PRBCS,None,carevue,1
1,5751,PRBCs,None,carevue,1
2,6256,Platelet Count,None,carevue,1
3,7597,PRBC,None,carevue,1
4,4197,RBC(3.6-6.2),Heme/Coag,carevue,1
5,3789,Platelet (150-440),Heme/Coag,carevue,1
6,833,RBC,Hematology,carevue,1
7,828,Platelets,Hematology,carevue,1
8,42794,Pheresis - Out,None,carevue,1
9,44879,Pheresis Output,None,carevue,1


## Question 1: What tables and features differentiate blood-transfusion-related and non-blood-transfusion-related admittances?


Potentially relevant tables:
* INPUTEVENTS_CV, INPUTEVENTS_MV, OUTPUTEVENTS: https://mimic.mit.edu/docs/iii/about/io/ 
* chartevents, datetimeevents, procedureevents_mv, services

NB: Timeshifting - https://mimic.mit.edu/docs/iii/about/time/#date-shifting
* Time of day, Day of week, and season(winter, spring, summer, fall) preserved
* year, exact day of month, patient overlap in ICU not preserved

NB: MIMIC Schema - https://mit-lcp.github.io/mimic-schema-spy/index.html 


In [67]:
# get the types of items that can be administered to a patient
with open(f"{query_path}/types_of_items_cv.sql", 'r') as file:
    query_cv = file.read()
with open(f"{query_path}/types_of_items_mv.sql", 'r') as file:
    query_mv = file.read()

results_cv = client.query(query_cv).to_dataframe()
print(results_cv)
results_mv = client.query(query_mv).to_dataframe()
results_mv

/home/kmorhun/.pyenv/versions/3.12.7/envs/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   itemid                label category dbsource  count
0   30179               PRBC's     None  carevue      1
1   30005  Fresh Frozen Plasma     None  carevue      1
2   30180    Fresh Froz Plasma     None  carevue      1
3   30001         Packed RBC's     None  carevue      1
4   30004        Washed PRBC's     None  carevue      1


/home/kmorhun/.pyenv/versions/3.12.7/envs/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,itemid,label,category,dbsource,count
0,227378,Patient Location,ADT,metavision,1
1,226543,Religion,ADT,metavision,1
2,226544,Language,ADT,metavision,1
3,226545,Race,ADT,metavision,1
4,226228,Gender,ADT,metavision,1
...,...,...,...,...,...
2987,220283,Resistance,Respiratory,metavision,1
2988,228382,Total Peripheral Resistance Index (TPRI) (NICOM),NICOM,metavision,1
2989,228183,SVRI (PiCCO)_OLD_1,PiCCO,metavision,1
2990,228185,SVRI (PiCCO),PiCCO,metavision,1


In [80]:
#found category 'Blood Products/Colloids'
with open(f"{query_path}/all_blood_products.sql", 'r') as file:
    query = file.read()

results = client.query(query).to_dataframe()
results

/home/kmorhun/.pyenv/versions/3.12.7/envs/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,label,itemid,category,dbsource,count
0,Packed Red Blood Cells,225168,Blood Products/Colloids,metavision,1
1,Platelets,225170,Blood Products/Colloids,metavision,1
2,Cryoprecipitate,225171,Blood Products/Colloids,metavision,1
3,Cell Saver,225173,Blood Products/Colloids,metavision,1
4,Hetastarch (Hespan) 6%,225174,Blood Products/Colloids,metavision,1
5,PACU Packed RBC Intake,227070,Blood Products/Colloids,metavision,1
6,PACU Platelet Intake,227071,Blood Products/Colloids,metavision,1
7,PACU FFP Intake,227072,Blood Products/Colloids,metavision,1
8,Fresh Frozen Plasma,220970,Blood Products/Colloids,metavision,1
9,IV Immune Globulin (IVIG),227530,Blood Products/Colloids,metavision,1


In [68]:
# as per our esteemed clinician's advice, we will use the following blood products:
chosen_blood_products_mv = (225168, 225170, 225171, 227070, 227071, 227072, 220970, 227532, 226367, 226368, 226369, 226371)
chosen_blood_products_cv = (30179, 30001, 30004, 30005, 30180)
with open(f"{query_path}/count_blood_inputs.sql", 'r') as file:
    query = file.read()

results = client.query(query).to_dataframe()
results

/home/kmorhun/.pyenv/versions/3.12.7/envs/transfusion/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,TOTALAMOUNTUOM,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE
0,3247771,51121,183688,214425,2124-05-23 03:00:00,2124-05-23 04:11:00,225825,4.733333,ml,4.000000,...,ml,0,0,0,Paused,None,None,NaT,38.000000,4.000000
1,203290,77427,163354,244787,2139-11-23 12:30:00,2139-11-23 12:37:00,225825,31.689280,ml,271.622400,...,ml,0,0,0,Changed,None,None,NaT,235.406110,271.840000
2,202734,77427,163354,244787,2139-11-22 13:30:00,2139-11-22 13:44:00,225825,2.537849,ml,10.876496,...,ml,0,0,0,Changed,None,None,NaT,244.902440,10.875465
3,203472,77427,163354,244787,2139-11-25 08:00:00,2139-11-25 11:00:00,225825,6.131904,ml,2.043968,...,ml,0,0,0,Changed,None,None,NaT,249.227830,2.038800
4,1461355,90650,154202,228989,2129-04-08 11:47:00,2129-04-08 21:04:00,225825,46.444120,ml,5.002957,...,ml,0,0,0,Stopped,None,None,NaT,71.292130,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,578946,90120,117195,282666,2186-03-04 08:00:00,2186-03-04 09:55:00,225910,15.333335,mg,8.000001,...,ml,0,0,0,Stopped,None,None,NaT,80.000000,8.000000
96,2927039,55477,184325,204235,2157-11-16 09:10:00,2157-11-16 13:29:00,225910,34.533338,mg,8.000001,...,ml,0,0,0,Stopped,None,None,NaT,80.000000,8.000000
97,1742421,25235,182008,251047,2167-06-20 22:00:00,2167-06-21 05:56:00,225910,79.360000,mg,10.003361,...,ml,0,0,0,FinishedRunning,None,None,NaT,79.360001,10.003360
98,966235,46034,199614,245855,2168-07-31 06:38:00,2168-07-31 16:35:00,225910,79.599998,mg,8.000000,...,ml,0,0,0,FinishedRunning,None,None,NaT,79.599998,8.000000
